<h1><center>ITenacity</center></h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections

from sklearn.linear_model import LinearRegression
from sklearn import metrics

## Предобработка данных

Загрузка датасетов

In [2]:
df_train = pd.read_csv('CUP_IT_train_data.csv', sep = ',', index_col = 0,header=133)
df_test = pd.read_csv('CUP_IT_test_data.csv', sep = ',', index_col = 0,header=133)

C:\Users\aleks\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Удаляем ненужные столбцы

In [3]:
df_train=df_train.drop(['cl_balance', 'mf_volume','ml_balance','sa_volume','cu_empl_level','cu_empl_area',
                        'dlk_cob_date', 'big_city', 'cu_gender', 'payroll_f', 'cur_quantity_mort', 'cur_quantity_deposits',
                        'cur_quantity_accounts', 'cur_quantity_mf','pl_balance', 'td_volume', 'dc_cash_spend_v', 'dc_cash_spend_c',
                        'cc_cash_spend_v', 'cc_cash_spend_c', 'dc_pos_spend_v', 'dc_pos_spend_c', 'cc_pos_spend_v', 'cc_pos_spend_c',
                        'ca_f', 'cur_qnt_sms','standalone_payroll_dc_f', 'standalone_nonpayroll_dc_f', 'salary', 'cu_mob',
                        'cu_empl_cur_dur_m','is_married'], axis=1)
df_test=df_test.drop(['cl_balance', 'mf_volume','ml_balance','sa_volume','cu_empl_level','cu_empl_area',
                      'dlk_cob_date', 'big_city', 'cu_gender', 'payroll_f','cur_quantity_mort', 'cur_quantity_deposits',
                      'cur_quantity_accounts', 'cur_quantity_mf','pl_balance', 'td_volume', 'dc_cash_spend_v', 'dc_cash_spend_c',
                      'cc_cash_spend_v', 'cc_cash_spend_c', 'dc_pos_spend_v', 'dc_pos_spend_c', 'cc_pos_spend_v', 'cc_pos_spend_c',
                      'ca_f', 'cur_qnt_sms','standalone_payroll_dc_f', 'standalone_nonpayroll_dc_f', 'salary', 'cu_mob', 'cu_empl_cur_dur_m',
                      'is_married','rank'], axis=1)

In [5]:
df_test

,gi_smooth_3m,cu_education_level,cur_quantity_pl,cur_quantity_cc,cur_quantity_dc,cur_quantity_saccounts,cc_balance,ca_volume,rc_session_qnt_cur_mon,active,standalone_dc_f,cu_age,cu_eduaction_level
cif_id,,,,,,,,,,,,,
50514,17.898438,03. Высшее,1,0,1,0,NaN,23.05368,3,1,1,48.0,3
50514,24.355419,03. Высшее,1,0,1,0,NaN,3.11064,3,1,1,47.0,3
50514,20.534134,03. Высшее,1,0,1,0,NaN,13.08216,4,1,1,48.0,3
50514,26.477938,03. Высшее,1,0,1,0,NaN,19.72488,2,1,1,47.0,3
50514,19.623962,03. Высшее,1,0,1,0,NaN,30.06792,1,1,1,48.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195116,215.977387,03. Высшее,0,2,1,0,-8870.57784,311.78016,0,1,0,35.0,3
195116,171.179781,03. Высшее,0,2,0,0,-9124.37136,NaN,0,1,0,35.0,3
195116,195.262540,03. Высшее,0,2,1,0,-8813.44440,1452.00000,0,1,0,35.0,3


Заменяем пустые заначения в категариальном признаке на наиболее вероятное

In [6]:
def replace_most_common(x):
    letter_cnt = collections.Counter(x)
    fil=letter_cnt.most_common(1)[0][0]
    return fil    
df_train['cu_education_level'].fillna(replace_most_common(df_train.cu_education_level), inplace = True)
df_test['cu_education_level'].fillna(replace_most_common(df_test.cu_education_level), inplace = True)

Кодирование категориальных признаков

In [7]:
def cod_categor(data,feature):
    for i in data[feature].unique():
        data[feature+'='+i]=(data[feature]==i).astype(float)
cod_categor(df_train,'cu_education_level')
df_train.drop(['cu_eduaction_level'], axis=1, inplace = True)
df_train.drop(['cu_education_level'], axis=1, inplace = True)
cod_categor(df_test,'cu_education_level')
df_test.drop(['cu_eduaction_level'], axis=1, inplace = True)
df_test.drop(['cu_education_level'], axis=1, inplace = True)

Заменяем пустые значения на медианные

In [8]:
df_train['cc_balance'].fillna(df_train.cc_balance.median(), inplace = True)
df_train['ca_volume'].fillna(df_train.ca_volume.median(), inplace = True)
df_test['cc_balance'].fillna(df_test.cc_balance.median(), inplace = True)
df_test['ca_volume'].fillna(df_test.ca_volume.median(), inplace = True)

Удаляем выбросы (только в возрасте (он был -100))

In [9]:
def del_em(data,feature):
    first_quartile = data[feature].describe()['25%']
    third_quartile = data[feature].describe()['75%']
    iqr = third_quartile - first_quartile
    data = data[(data[feature] > (first_quartile - 3* iqr)) &
                (data[feature] < (third_quartile + 3* iqr))]
    return data
df_train=del_em(df_train,'cu_age')
df_test=del_em(df_test,'cu_age')

## Строим модель для прогноза

In [10]:
y_train=df_train[['gi_smooth_3m']]
X_train=df_train.iloc[:,1:]

y_test=df_test[['gi_smooth_3m']]
X_test=df_test.iloc[:,1:]

regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [11]:
pred=list(pd.DataFrame(y_pred)[0])
test=list(y_test['gi_smooth_3m'])
id_=list(df_test.reset_index()['cif_id'])
df_pred  = pd.DataFrame({'cif_id': id_,
                         'cltv': pred})

In [12]:
print('Mean Absolute Error:', metrics.mean_absolute_error(test, pred))

Mean Absolute Error: 24.16607242978641


In [13]:
df_pred=df_pred[['cif_id','cltv']].groupby('cif_id').sum()
# df_pred.to_csv('CUP_IT_predictions_ITenacity.csv')

In [14]:
df_pred

,cltv
cif_id,
1000,362.038964
1003,458.135156
1006,82.942619
1008,43.416983
1012,129.238835
...,...
511518,293.714055
511519,256.150393
511523,204.670869


In [19]:
df_test.reset_index()[['cif_id','gi_smooth_3m']].groupby('cif_id').sum()

,gi_smooth_3m
cif_id,
1000,323.642757
1003,450.702379
1006,14.598418
1008,8.081938
1012,44.656817
...,...
511518,302.272926
511519,232.425340
511523,17.648986
